# h5/hdf5 to AnnData

In [2]:
# Import modules
import numpy as np
import scanpy as sc
import pandas as pd
import anndata as ad
import session_info
from scipy.sparse import csr_matrix
import h5py

In [3]:
# Information of the session

sc.settings.verbosity = 3

session_info.show()

### Check content of data.h5

In [8]:
# Identify files
import h5py

with h5py.File(file_path, 'r') as archivo:
    def listar_objetos(name, obj):
        print(name)

    archivo.visititems(listar_objetos)

GRCh38
GRCh38/barcodes
GRCh38/data
GRCh38/gene_names
GRCh38/genes
GRCh38/indices
GRCh38/indptr
GRCh38/shape


The content of .h5 doesn't have a defined name. In the next step, you need to specify paths to the data (values ​​other than 0), column indexes (position values), indptr (start and end of row), and shape (shape of the matrix). 

You also need to specify paths to the barcodes or cell_names, and the genes.

In [9]:
# Keys
data_key = "GRCh38/data"
indices_key = "GRCh38/indices"
indptr_key = "GRCh38/indptr"
shape_key = "GRCh38/shape"
barcodes_key = "GRCh38/barcodes"
gene_names_key = "GRCh38/gene_names"

### Check the format of your data

If you have 10x Genomics data, set the format_10x variable to **True**

In [4]:
format_10x = False
file_path = "/Users/rafaelsalgueroraigon/Downloads/GSM3489183_IPF_01_filtered_gene_bc_matrices_h5.h5"

In [10]:
# 10x genomics
if format_10x:
    print("This data is in 10x Genomics format")
    adata = sc.read_10x_h5(file_path)
else:
    print("This data is not in 10x Genomics format")
    with h5py.File(file_path, "r") as f:
        data = f[data_key][:]
        indices = f[indices_key][:]
        indptr = f[indptr_key][:]
        shape = tuple(f[shape_key][:])[::-1]  

        X = csr_matrix((data, indices, indptr), shape=shape)

        cell_names = [b.decode("utf-8") for b in f[barcodes_key][:]]
        gene_names = [g.decode("utf-8") for g in f[gene_names_key][:]]

    obs = pd.DataFrame(index=cell_names)
    var = pd.DataFrame(index=gene_names)
    adata = sc.AnnData(
        X=X,  
        obs=obs,
        var=var
    )

print(adata)

This data is not in 10x Genomics format
AnnData object with n_obs × n_vars = 2253 × 33694


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/anndata/_core/anndata.py:1756: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [11]:
# Save anndata

adata.write ("adata.h5ad") # compression="gzip" (optional) 

## Reference data:

https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi 

Sample: GSM3489183

